In [ ]:
#! pip install -U openai-whisper

In [2]:
# cd drive/MyDrive/sonny

/content/drive/MyDrive/sonny


In [4]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from tqdm.notebook import tqdm


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
from torch.utils.data import Dataset
from torch.nn import functional as F
import random
import torch
import numpy as np
import pandas as pd
import soundfile as sf

In [7]:
class customAudioDataset(Dataset):
      def __init__(self, csv_path):
            self.data = pd.read_csv(csv_path, header=[0, 1])
            self.text_data = self.data['text_data'][' '].values
            self.wav_dir = self.data['wav_dir'][' '].values

      def __len__(self):
            return len(self.data)
          
      def __getitem__(self, idx):
            try:
                audio_input, sample_rate = sf.read(self.wav_dir[idx])
                audio = whisper.pad_or_trim(audio_input)
                audio = torch.tensor(audio, dtype = torch.float32)
                mel = whisper.log_mel_spectrogram(audio)
            except:
                return None
            return (mel, self.text_data[idx])

In [9]:
def my_collate(batch):
    len_batch = len(batch)
    batch = list(filter(lambda x: x is not None, batch))

    if len_batch > len(batch):                
        db_len = len(dataset)
        diff = len_batch - len(batch)
        while diff != 0:
            a = dataset[np.random.randint(0, db_len)]
            if a is None:                
                continue
            batch.append(a)
            diff -= 1

    return torch.utils.data.dataloader.default_collate(batch)

In [ ]:
dataset = customAudioDataset('balance_train.csv')

In [10]:
loader = torch.utils.data.DataLoader(dataset, collate_fn=my_collate, batch_size=16)

In [11]:
model = whisper.load_model("base")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 126MiB/s]


Model is multilingual and has 71,825,920 parameters.


In [12]:
options = whisper.DecodingOptions(language="ko", without_timestamps=True)

In [13]:
hypotheses = []
references = []

for mels, texts in tqdm(loader):
    mels = mels.to('cuda')
    results = model.decode(mels, options)
    hypotheses.extend([result.text for result in results])
    references.extend(texts)

  0%|          | 0/4377 [00:00<?, ?it/s]

In [ ]:
data = pd.DataFrame(dict(hypothesis=hypotheses, reference=references))
data